In [1]:
import os
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
from glob import glob
import pandas as pd

from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import Docx2txtLoader
from langchain_community.document_loaders import UnstructuredPowerPointLoader
from langchain_community.document_loaders import CSVLoader

from typing import Any, Dict, Iterator, List, Optional, Sequence, Union

In [2]:

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# langsmith monitoring environment setting
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ['LANGCHAIN_API_KEY'] = 'ls__e15765918790425c8ef8ccdde35deed9'
os.environ['LANGCHAIN_PROJECT'] = 'self-querying-chroma-test01'


embedding 

In [3]:
embeddings = HuggingFaceEmbeddings(
                    model_name="intfloat/multilingual-e5-large",
                    model_kwargs={'device': 'cuda'}, # streamlit에서는 gpu 없음
                    encode_kwargs={'normalize_embeddings': True}
                )

### 데이터 불러오기

참고 : 
- langchain_community/document_loaders/CSVLoader : https://github.com/langchain-ai/langchain/blob/master/libs/community/langchain_community/document_loaders/csv_loader.py
- langchain_core/document_loaders/base.py : https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/document_loaders/base.py

In [4]:
import csv
import pandas as pd


def get_text(
        files_path : str, 
        column_list : Sequence[str] = (), 
        source_dict : Sequence[str] = {},
        metadata_columns : Sequence[str] = (),
    ):
    # column_list가 있으면 column list 열만 page_content에 넣음. 없으면 모든 열을 page_content에 넣음.

    file_list = glob(files_path + '/*')
    doc_list = []

    for file in file_list:
        if file.endswith('.csv'):
            with open(file, newline='') as csvfile:
                df = pd.read_csv(csvfile)
                documents = list(df_to_doc(df, files_path, column_list, source_dict, metadata_columns))
        elif file.endswith('.pdf'):
            pass
        doc_list.extend(documents)
        
    return doc_list
    
    
def df_to_doc(
        df : pd.DataFrame, 
        file_path : str, 
        column_list : Sequence[str] = (),
        source_dict : Dict = {},
        metadata_columns : Sequence[str] = (),
    ) -> Iterator[Document]:

    if not column_list: 
        column_list = df.columns.to_list()
        print(column_list)

    # if True in df[column_list].isna().any().to_list():
        # raise ValueError("The Required Column has empty value. Cannot process") 
    
    df.fillna('', inplace=True)

    # Joining the columns with a 'ᴥ'
    
    # column명 없이
    df['content'] = df.apply(lambda row: 'ᴥ'.join(row[column_list]), axis=1)
    
    # column명 넣어서
    # df['content'] = df.apply(lambda row: 'ᴥ'.join([f"{col};{val}" for col, val in row[column_list].items()]), axis=1)
    
    
    for _, data in df.iterrows():
        metadata = dict()

        for key, value in source_dict.items():
            metadata[key] = value
            
        for col in metadata_columns:
            metadata[col] = data[col]

        metadata['source'] = file_path
            
        yield Document(page_content=data['content'], metadata=metadata)



#### chroma db용 데이터 생성 및 chroma db 저장, 불러오기

In [5]:
tmp = get_text('./files/terms', 
               column_list=['단어', '설명', '출처', '정책명'], 
               source_dict={'category': '정보제공'}, 
               metadata_columns=['단어', '정책명'])

process_df = pd.read_csv('./files/기관_2024-03-04 - 기관_2024-03-04.csv')
tmp2 = df_to_doc(process_df, 
                './files', 
                column_list = ['정책명', '신청 절차', '제출 서류', '심사 발표', '신청 사이트 주소', '참고 사이트 주소1', '참고 사이트 주소2'],
                source_dict={'category':'절차문의'},
                metadata_columns=['정책명']
                )
tmp2 = list(tmp2)
tmp3 = get_text('./files/whole',
                column_list=['정책명', '답변', 'source'],
                source_dict={'category': '정보제공'},
                metadata_columns=['정책명']
                )
tmp.extend(tmp2)
tmp.extend(tmp3)
tmp
# tmp = get_text('./files/terms', column_list=['단어', '설명'])
# tmp = get_text('./files/terms')



/tmp/ipykernel_1138328/344558436.py:43: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('', inplace=True)
/tmp/ipykernel_1138328/344558436.py:43: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('', inplace=True)


[Document(page_content='기준금리ᴥ한국은행의 최고 결정기구인 금융통화위원회에서 연 8회에 걸쳐 회의를 통해서 결정하는 금리.ᴥhttps://www.moef.go.kr/sisa/dictionary/detail?idx=730ᴥ청년전용 버팀목전세자금', metadata={'category': '정보제공', '단어': '기준금리', '정책명': '청년전용 버팀목전세자금', 'source': './files/terms'}),
 Document(page_content='세대주ᴥ주민등록상의 세대란 주거 및 생계를 같이 하는 집단이며, 세대주는 그 세대를 대표하는 자로서, 주민의 신고에 의하여 선정이 가능합니다.ᴥhttp://www.gangdong.go.kr:8080/web/dongrenew/bbsamsa3/faq_amsa3/27395ᴥ청년전용 버팀목전세자금', metadata={'category': '정보제공', '단어': '세대주', '정책명': '청년전용 버팀목전세자금', 'source': './files/terms'}),
 Document(page_content='주택도시보증공사ᴥ주택도시보증공사(HUG)는 주택 관련 보증업무 및 정책사업의 수행과 주택도시기금의 효율적 운용·관리를 통하여 국민 삶의 질 향상에 이바지하기 위한 목적으로 주택도시기금법에 근거하여 설립된 국토교통부 산하 공기업이다.ᴥhttps://namu.wiki/w/%EC%A3%BC%ED%83%9D%EB%8F%84%EC%8B%9C%EB%B3%B4%EC%A6%9D%EA%B3%B5%EC%82%ACᴥ청년전용 버팀목전세자금', metadata={'category': '정보제공', '단어': '주택도시보증공사', '정책명': '청년전용 버팀목전세자금', 'source': './files/terms'}),
 Document(page_content='전세금안심대출보증ᴥ전세보증금반환보증 정의:\n임차인에게 전세보증금반환(전세보증금반환보증)과 금융기관에 전세자금대출의 원리금 상환(전세자금대출특약보증)을 

In [6]:
tmp4 = list(df_to_doc(process_df,
                      './files',
                      column_list = ['정책명', '정책 소개', '연령', '전공', '취업 상태', '학력', '거주지 및 소득', '추가 단서 사항', '참여 제한 대상'],
                      source_dict = {'category':'자격요건'},
                      metadata_columns=['정책명', '연령', '전공', '취업 상태', '학력', '거주지 및 소득', '참여 제한 대상']
                      ))

In [7]:
len(tmp)

1131

In [13]:
vectorstore = Chroma.from_documents(tmp, embeddings, persist_directory='./chroma_db2')

In [4]:
vectorstore = Chroma(persist_directory="./chroma_db2", embedding_function=embeddings)

## Self-Querying 사용

self-querying 설명
- https://python.langchain.com/docs/modules/data_connection/retrievers/self_query
- integrations : https://python.langchain.com/docs/integrations/retrievers/self_query/
- chroma self-querying : https://github.com/langchain-ai/langchain/blob/master/docs/docs/integrations/retrievers/self_query/chroma_self_query.ipynb

source code :
- from_llm source : https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/retrievers/self_query/base.py

Chroma.as_retriever source code 
- https://github.com/langchain-ai/langchain/blob/master/libs/community/langchain_community/vectorstores/chroma.py
- as_retriever source : https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/vectorstores.py

chroma self query source code
- https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/retrievers/self_query/chroma.py

langchain chroma documentation : https://api.python.langchain.com/en/latest/vectorstores/langchain_community.vectorstores.chroma.Chroma.html#langchain_community.vectorstores.chroma.Chroma.as_retriever

In [5]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import OpenAI, ChatOpenAI


In [9]:
retriever = vectorstore.as_retriever(search_type="mmr", verbose=True)
type(retriever)

langchain_core.vectorstores.VectorStoreRetriever

In [9]:
metadata_field_info = [
    AttributeInfo(
        name="category",
        description="classification according to Query intent",
        type="string",
    ),
    AttributeInfo(
        name="정책명",
        description="The policy name related to passage",
        type="integer",
    ),
    AttributeInfo(
        name="단어",
        description="The terms explained by the passage",
        type="string",
    ),
]
document_content_description = "Explanation of terms related to policy"
llm = ChatOpenAI(
            model_name="gpt-3.5-turbo",
            temperature=0,
            )
retriever = SelfQueryRetriever.from_llm(
    llm, vectorstore, document_content_description, metadata_field_info, verbose=True
    # llm, vectorstore.as_retriever(search_type='mmr', verbose=True), document_content_description, metadata_field_info, verbose=True
)


self query retreiver 
- source code : https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/retrievers/self_query/base.py
- document : https://api.python.langchain.com/en/latest/retrievers/langchain.retrievers.self_query.base.SelfQueryRetriever.html
- search type : https://api.python.langchain.com/en/latest/vectorstores/langchain_community.vectorstores.chroma.Chroma.html#langchain_community.vectorstores.chroma.Chroma.as_retriever

In [12]:
print(type(retriever)) # 
print(dir(retriever))

<class 'langchain.retrievers.self_query.base.SelfQueryRetriever'>
['Config', 'InputType', 'OutputType', '__abstractmethods__', '__annotations__', '__class__', '__class_getitem__', '__class_vars__', '__config__', '__custom_root_type__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__exclude_fields__', '__fields__', '__fields_set__', '__format__', '__ge__', '__get_validators__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__include_fields__', '__init__', '__init_subclass__', '__iter__', '__json_encoder__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__or__', '__orig_bases__', '__parameters__', '__post_root_validators__', '__pre_root_validators__', '__pretty__', '__private_attributes__', '__reduce__', '__reduce_ex__', '__repr__', '__repr_args__', '__repr_name__', '__repr_str__', '__rich_repr__', '__ror__', '__schema_cache__', '__setattr__', '__setstate__', '__signature__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__try_update_for

In [10]:
# retriever.get_relevant_documents("금리가 뭐야?")
# retriever.get_relevant_documents("금리가 뭐야?", metadata={'category':'정보제공'})
# retriever.get_relevant_documents("[category:정보제공] 금리가 뭐야?")
# retriever.get_relevant_documents("[category:절차문의] 금리가 뭐야?")

# retriever.get_relevant_documents("[category:정보제공] 정책 중에 전세 자금 대출 관련 정책이 있으면 추천해줘.")
retriever.get_relevant_documents("청년도약계좌 자격요건을 알려줘.")

[Document(page_content='청년도약계좌ᴥ가입신청 시 제출할 서류는 없습니다.단, 가입요건 충족여부 확인 과정에서 전산정보만으론 정보확인이 불가한 경우, 아래 서류제출을 요청할 수 있습니다.가입요건서류제출 대상자제출서류나이만34세 초과 병역이행기간 확정 불가자군경력증명서, 병적증명서*발급 : 정부24>기타요청사항지방병무청, 병무청 콜센터개인소득육아휴직급여 및 육아휴직수당 수령자ㆍ일반근로자: 육아휴직급여 지급결정 통지서, 신분증 사본(주민번호 13자리 확인 必)ㆍ공무원: 휴직증명서, 급여명세서(육아휴직수당 내역 포함), 신분증 사본(주민번호 13자리 확인 必)가구원확정주민등록표등본에 신청인과의 관계가 불분명한 세대원이 있는 신청자가족관계증명서* 발급처 : 법원 전자가족관계등록시스템최신화①등본에 조회된 가구원이 사망 등으로 제외가 필요한 경우②신청자가 부양중인 조(외조)부모가 있는 신청자상세서류 별도확인소득--ᴥhttps://blog.naver.com/mltmkr/223359281452', metadata={'category': '정보제공', 'source': './files/whole', '정책명': '청년도약계좌'}),
 Document(page_content='청년도약계좌ᴥ청년도약계좌는 국세청에서 제공하는 전산 정보를 바탕으로 개인소득 요건 충족 여부를 확인합니다.※ 육아휴직직급여 및 육아휴직수당 외엔 별도 증빙서류 제출 불필요 개인소득 요건 확인의 각 기준은 아래와 같습니다.1.기준연도 : 계좌개설일 전년도 또는 전전년도(국세청에서 소득액 확정 전)2024.7월 가입신청 시, 2023년도 소득을 기준으로 가입요건 확인2024.6월 가입신청 시, 2022년도 소득을 기준으로 가입요건 확인2. 산정기준 : 총급여(근로소득), 종합소득, 사업소득(연금소득 및 종교인소득 포함)3. 가입요건 : 아래 3가지 조건을 동시 충족구분가입요건 상세1가입요건 확인 기준연도에 과세소득이 존재2가입요건 확인 기준연도에 총급여액이 7,500만원 이하3가입요건 확인 

In [19]:
# retriever.get_relevant_documents("[category:관련없음] 마산치매안심센터 증축을 위한 설계공모의 입선작을 창원이 뽑은 건 언제야")
# retriever.get_relevant_documents("마산치매안심센터 증축을 위한 설계공모의 입선작을 창원이 뽑은 건 언제야", metadata={'category': 'x'})    
# retriever.get_relevant_documents("마산치매안심센터 증축을 위한 설계공모의 입선작을 창원이 뽑은 건 언제야")

[Document(page_content='창원 청년 프로젝트ᴥ참가신청서 및 제출서류 작성 후 담당자 이메일로 송부(clouds@chari.re.kr)ᴥ□ 제출서류\n① 참가신청서 1부 \n② 사업계획서 1부 \n③ 개인정보동의서 1부\n④ 청렴이행서약서 1부ᴥㅁ 선발일정 및 방법(안)\n- 모집 및 신청 : 2.15.(목) ~ 3.13.(수)\n- 1차 심사(서면) : 3.14.(목)\n- 1차 심사 결과발표 : 3.15.(금)\n- 2차 심사(면접) : 3.23.(토)\n- 최종선정 결과공고 : 3.25.(월)\n ※ 심사 합격자에 한하여 개별 통보(유선전화) \n※ 세부 일정은 추진사항에 따라 변경될 수 있음ᴥ-ᴥhttps://www.changwon.go.kr/youth/05085/05105.web?amode=view&idx=77ᴥ', metadata={'category': '절차문의', '정책명': '창원 청년 프로젝트'}),
 Document(page_content='2024년 첫만남 이용권ᴥ□ 행정복지센터 및 복지로24, 정부24ᴥ-ᴥ-ᴥ-ᴥhttps://www.mohw.go.kr/board.es?mid=a10411010100&bid=0019ᴥ', metadata={'category': '절차문의', '정책명': '2024년 첫만남 이용권'}),
 Document(page_content='산모·신생아 건강관리 지원사업ᴥ방문, 인터넷ᴥ사회보장급여(사회서비스이용권) 신청(변경)서 또는 임신서비스 통합처리 신청서 각 1부ᴥ-ᴥ-ᴥ-ᴥ-', metadata={'category': '절차문의', '정책명': '산모·신생아 건강관리 지원사업'}),
 Document(page_content='경남창조경제혁신센터 신규입주기업 모집ᴥ담당자 E-MAIL 제출 : hjyh909@ccei.krᴥ-ᴥ-ᴥhttps://ccei.creativekorea.or.kr/gyeongnam/allim/allim_view.do?div_code=1&no=13695ᴥ-ᴥ-', metadat

In [12]:
metadata_field_info = [
    AttributeInfo(
        name="category",
        # description="classification according to Query intent",
        description="절차문의", # 기능을 쪼개서 챗봇을 사용할 경우. 이렇게 쓰면(기능별로 카테고리명을 써주면 ) 더 잘 나오는 것 같기도..
        type="string",
    ),
    AttributeInfo(
        name="정책명",
        description="Policy name of the procedure indicated in the passage",
        type="integer",
    )
]
document_content_description = "Explanation of the procedure related to policy"
llm = ChatOpenAI(
            model_name="gpt-3.5-turbo",
            temperature=0,
            )
retriever = SelfQueryRetriever.from_llm(
    llm, vectorstore, document_content_description, metadata_field_info, verbose=True
)

In [12]:
# retriever.get_relevant_documents("기후동행카드는 어떻게 신청해?")
# retriever.get_relevant_documents("[category:절차문의] 기후동행카드는 어떻게 신청해?")
# retriever.get_relevant_documents("[category:절차문의][정책명:기후동행카드] 기후 동행 카드는 어떻게 신청해?")

# retriever.get_relevant_documents("[category:절차문의] 국민취업제도는 어떻게 신청해?")
# retriever.get_relevant_documents("[category:절차문의] 국취제는 어떻게 신청해?")
# retriever.get_relevant_documents("[category:절차문의][정책명:국민취업지원제도] 국취제는 어떻게 신청해?")

# retriever.get_relevant_documents("청년도약계좌는 어떻게 신청해?")
# retriever.get_relevant_documents("청년도약계좌는 어떻게 신청해?", metadata={'category':'절차문의', '정책명':'청년도약계좌'})
# retriever.get_relevant_documents("[category:절차문의] 청년도약계좌는 어떻게 신청해?")
# retriever.get_relevant_documents("[category:정보제공] 청년도약계좌는 어떻게 신청해?")
# retriever.get_relevant_documents("[category:절차문의][정책명:청년도약계좌] 청년도약계좌는 어떻게 신청해?")

# retriever.get_relevant_documents("버팀목전세자금대출은 어떻게 신청해?")
retriever.get_relevant_documents("버팀목전세자금대출은 뭐야?")

[Document(page_content='청년전용 버팀목전세자금ᴥ청년전용 버팀목전세자금 정책 소개: 전세자금이 부족한 청년들에게 청년전용 버팀목 전세자금을 대출ᴥhttps://www.youthcenter.go.kr/youngPlcyUnif/youngPlcyUnifDtl.do?pageIndex=1&frameYn=&bizId=R2024010918589&dtlOpenYn=&plcyTpOpenTy=&plcyCmprInfo=&srchWord=&chargerOrgCdAll=&srchAge=&trgtJynEmp=&trgtJynEmp=&srchSortOrder=1&pageUnit=12', metadata={'category': '정보제공', 'source': './files/whole', '정책명': '청년전용 버팀목전세자금'}),
 Document(page_content='청년전용 버팀목전세자금ᴥ○ 대출 실행 후 아래의 기일 중 늦을 날로부터 14일 이내에 대출계약철회 가능 1) 대출계약서류를 제공받은 날 2) 대출계약체결일 3) 대출실행일 4) 사후자가산심사결과 부적격 확정통지일 - 대출계약 철회는 채무자가 철회기한 이내에 원금과 이자 및 부대비용을 전액 반환한 때에 효력이 발생○ 대출계약 철회권의 효력이 발생한 이후에는 철회권 행사 취소 불가ᴥhttps://nhuf.molit.go.kr/FP/FP05/FP0502/FP05020303.jsp?gotoPage=1', metadata={'category': '정보제공', 'source': './files/whole', '정책명': '청년전용 버팀목전세자금'}),
 Document(page_content='집단전세보증ᴥ공사의 사전승인을 받은 임대사업장 입주예정자의 임차중도금 및 잔금 대출을 위한 공사의 담보(보증서) 제공 시 소득 및 부채 수준에 따른 상환능력 평가를 생략할 수 있는 상품ᴥhttps://www.hf.go.kr/ko/sub02/sub02_01_03.doᴥ청년전용 버팀목전세자금', metadata={'catego

In [14]:
retriever.get_relevant_documents("국민취업제도 자격요건을 알려줘.")

[Document(page_content='국민취업지원제도ᴥ주민등록표에 등재된① 신청인 본인② 신청인의 배우자③ 신청인의 1촌 이내 직계혈족(부모, 자녀)으로 한정해 판단합니다.ᴥhttps://1350.moel.go.kr/home/hp/data/faqView.do?faq_idx=1000000722', metadata={'category': '정보제공', 'source': './files/whole', '정책명': '국민취업지원제도'}),
 Document(page_content='국민취업지원제도ᴥ참여가 어렵습니다.국민취업지원제도는 취업을 지원하는 목적의 사업으로서, 취업을 희망하는 분들 중근로능력과 구직의사가 있는 경우에 참여 가능합니다.ᴥhttps://1350.moel.go.kr/home/hp/data/faqView.do?faq_idx=1000000728', metadata={'category': '정보제공', 'source': './files/whole', '정책명': '국민취업지원제도'}),
 Document(page_content='국민취업지원제도ᴥⅠ유형으로 1)요건심사형, 2)선발형 청년층, 3)선발형 비경제활동인구 중 하나에 해당되면서, 수급자격이 인정된 이후에는 매월 2회 이상 구직활동 의무를 이행해야 합니다. 구분취업지원서비스 수급요건가구단위 소득요건가구원 재산요건취업경험 요건1) 요건 심사형구직능력 및 의사가 있는 15~69세로 미취업 상태중위소득 60% ↓4억원(15~34세 청년은 5억원) ↓최근 2년 이내100일 또는 800시간 ↑2) 선발형 청년층구직능력 및 의사가 있는 15~34세로 미취업 상태중위소득 120% ↓5억원 ↓3) 선발형 비경활구직능력 및 의사가 있는 15~69세로 미취업 상태중위소득 60% ↓4억원 ↓최근 2년 이내100일 또는 800시간↓ᴥhttps://www.kua.go.kr/uapae010/selectQstn.do?pageIndex=1&pageUnit=10&srchSecd=A&srchTy=&srchKwrd=', met

In [27]:
pdf = pd.read_csv('./files/기관_2024-03-04 - 기관_2024-03-04.csv')
print(pdf.columns.to_list())

['Unnamed: 0', '번호', '정책 ID', '정책 일련번호', '기관 및 지자체 구분', '정책명', '정책 소개', '지원 내용', '지원 규모', '사업 운영 기간', '사업 신청 기간 반복 구분 코드', '사업 신청 기간', '연령', '전공', '취업 상태', '특화 분화', '학력', '거주지 및 소득', '추가 단서 사항', '참여 제한 대상', '신청 절차', '제출 서류', '심사 발표', '신청 사이트 주소', '참고 사이트 주소1', '참고 사이트 주소2', '주관 부처명', '주관 부처 담당자 이름', '주관 부처 담당자 연락처', '운영 기관명', '운영 기관 담당자 이름', '운영 기관 담당자 연락처', '기타 사항', '정책 분야 코드', '정책 분야', '온통 청년 url']


In [ ]:
pdf[['정책명', '정책 소개', '연령', '전공', '취업 상태', '학력', '거주지 및 소득', '추가 단서 사항', '참여 제한 대상']].head()
pdf[(pdf['전공'] != '-') | (pdf['취업 상태'] != '-')][['정책명', '정책 소개', '연령', '전공', '취업 상태', '학력', '거주지 및 소득', '추가 단서 사항', '참여 제한 대상']]

In [25]:
ddf = pdf[['정책명', '신청 절차', '제출 서류', '심사 발표', '신청 사이트 주소', '참고 사이트 주소1', '참고 사이트 주소2']]

In [26]:
ddf[(ddf['심사 발표'] != '-') & (ddf['심사 발표'].notna())].head()
ddf[ddf['정책명'] == '청년도약계좌']

,정책명,신청 절차,제출 서류,심사 발표,신청 사이트 주소,참고 사이트 주소1,참고 사이트 주소2
427,청년도약계좌,[매월]\n가입신청 - 취급은행 앱\n[신청 후 약 2주]\n가입심사 – 서민금융진...,NaN,□ 개인 및 가구소득 확인 세부 절차 (약 2주 소요)\n① 나이 및 개인소득 확인...,취급은행,https://ylaccount.kinfa.or.kr,https://www.kinfa.or.kr/customerService/boardF...


더 보면 좋을 것
- https://github.com/langchain-ai/langchain/blob/master/cookbook/self_query_hotel_search.ipynb
- raptor : https://github.com/langchain-ai/langchain/blob/master/cookbook/RAPTOR.ipynb

chromadb 관련 

https://python.langchain.com/docs/integrations/vectorstores/chroma
https://docs.trychroma.com/reference/Client
https://github.com/chroma-core/chroma

https://gniogolb.tistory.com/18
https://opentutorials.org/module/6369/32376

In [17]:
import uuid
import chromadb
from chromadb.config import Settings
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

client = chromadb.HttpClient(
    host = '175.45.203.113',
    port = "1123",
    settings=Settings(allow_reset=True)
)
client.reset()  # resets the database
collection = client.create_collection("test_collection")

collection.add(
    ids=[str(uuid.uuid1())], metadatas={'testMetadata':'tempt'}, documents='test'
)

embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large",
    model_kwargs={"device": "cuda"},  # streamlit에서는 gpu 없음
    encode_kwargs={"normalize_embeddings": True},
)

# tell LangChain to use our client and collection name
db4 = Chroma(
    client=client,
    collection_name="test_collection",
    embedding_function=embeddings,
)
query = "What did the president say about Ketanji Brown Jackson"
docs = db4.similarity_search(query)
print(docs[0].page_content)

InvalidDimensionException: Embedding dimension 1024 does not match collection dimensionality 384

In [14]:
result = collection.query(
    query_texts=['test'],
    n_results=1,
)
print(result)

{'ids': [['c0dfa634-e234-11ee-84b0-f220af9362dc']], 'distances': [[0.0]], 'embeddings': None, 'metadatas': [[{'testMetadata': 'tempt'}]], 'documents': [['test']], 'uris': None, 'data': None}
